In [1]:
#!pip install kaggle

## Imports e Especificações de Libs

In [16]:
import pandas as pd 
import numpy as np 
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

## EDA

In [3]:
path_to_data = r"data/"

In [4]:
user_review = pd.read_csv(f'{path_to_data}animelist.csv')
anime_list = pd.read_csv(f'{path_to_data}anime.csv')
anime_description = pd.read_csv(f'{path_to_data}anime_with_synopsis.csv')

In [5]:
#How many users
len(user_review.user_id.unique())

325770

In [14]:
#Showing first 10 anime ratings
'''for index, row in user_review.iterrows():
    print(index,row)
    if(index > 10):
        break'''

'for index, row in user_review.iterrows():\n    print(index,row)\n    if(index > 10):\n        break'

In [8]:
anime_full = pd.merge(anime_list, anime_description[["MAL_ID","sypnopsis"]], how='inner', on="MAL_ID")

In [9]:
anime_description.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'sypnopsis'], dtype='object')

Missing Data

In [10]:
anime_full.isnull().sum().tail(5)

Score-4      0
Score-3      0
Score-2      0
Score-1      0
sypnopsis    8
dtype: int64

Choosing Features

In [17]:
anime_full.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1', 'sypnopsis'],
      dtype='object')

In [76]:
anime_full.sample(1)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,sypnopsis
12656,36774,Star Wars/Saigo no Jedai: Tokubetsu Eizou,5.1,"Sci-Fi, Parody",Unknown,『スター・ウォーズ／最後のジェダイ』特別映像,ONA,1,"Nov 8, 2017",Unknown,...,1.0,5.0,13.0,31.0,53.0,36.0,21.0,14.0,19.0,promotional video for Star Wars: The Last Jedi...


In [85]:
features = ["Name", "Genres", "Episodes","Studios","Rating","Score","Aired","sypnopsis"]

In [92]:
anime_full.sypnopsis.fillna(' ', inplace=True)
anime_full.isna().sum().sum()

0

In [106]:
featured = anime_full[features].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [109]:
featured = featured .replace(',','', regex=True)

In [110]:
featured

0        Cowboy Bebop_Action Adventure Comedy Drama Sci...
1        Cowboy Bebop: Tengoku no Tobira_Action Drama M...
2        Trigun_Action Sci-Fi Adventure Comedy Drama Sh...
3        Witch Hunter Robin_Action Mystery Police Super...
4        Bouken Ou Beet_Adventure Fantasy Shounen Super...
                               ...                        
16209    Daomu Biji Zhi Qinling Shen Shu_Adventure Myst...
16210    Mieruko-chan_Comedy Horror Supernatural_Unknow...
16211    Higurashi no Naku Koro ni Sotsu_Mystery Dement...
16212    Yama no Susume: Next Summit_Adventure Slice of...
16213    Scarlet Nexus_Action Fantasy_Unknown_Sunrise_U...
Length: 16214, dtype: object

In [111]:
vectorizer = TfidfVectorizer()

In [112]:
feature_vectors = vectorizer.fit_transform(featured)

In [115]:
print(feature_vectors)

  (0, 11232)	0.03725364267326765
  (0, 48521)	0.044271829492614094
  (0, 36769)	0.030660488632878506
  (0, 47957)	0.045463325869260246
  (0, 35615)	0.10376615923725806
  (0, 64488)	0.0947534033421958
  (0, 34195)	0.09568642819989229
  (0, 19356)	0.07132644034983315
  (0, 73930)	0.07859394126746425
  (0, 65063)	0.05018313341078838
  (0, 33073)	0.07654365625315983
  (0, 42935)	0.05930743645276868
  (0, 10874)	0.06015152002821227
  (0, 22745)	0.11657222994739565
  (0, 33573)	0.04263447759466045
  (0, 73894)	0.05040065026843234
  (0, 43149)	0.09866657845681602
  (0, 52738)	0.05300673373455562
  (0, 48199)	0.04327725394235146
  (0, 22062)	0.05816531921030628
  (0, 45696)	0.060244033116192315
  (0, 24863)	0.04574103423462512
  (0, 72138)	0.07859394126746425
  (0, 11123)	0.05701905813989433
  (0, 69576)	0.08916321757797344
  :	:
  (16213, 54834)	0.1420230302097039
  (16213, 59599)	0.11225097845781959
  (16213, 25432)	0.11689387634909258
  (16213, 24537)	0.09828180253554712
  (16213, 53004)	0.

Cosine Similarity

In [116]:
similarity = cosine_similarity(feature_vectors)

In [118]:
similarity.shape

(16214, 16214)

Getting the Anime Name

In [205]:
anime_name = input('Enter anime name : ')

In [206]:
title_list = anime_full.Name.to_list()

In [207]:
find_close_match = difflib.get_close_matches(anime_name, title_list)
anime_match = find_close_match[0]


In [213]:
index_of_anime = anime_full[anime_full.Name == anime_match].index.values[0]

In [209]:
index_of_anime

4216

In [210]:
similarity_score = list(enumerate(similarity[index_of_anime]))

In [211]:
most_similar_animes = sorted(similarity_score, key=lambda x:x[1], reverse=True)

In [212]:
print("Suggested Animes for your sunday night : \n")

for movie in most_similar_animes[:10]:
    print(anime_full[anime_full.index == movie[0]].Name.values[0])

Suggested Animes for your sunday night : 

Ring of Gundam
After War Gundam X
Mobile Suit Gundam Unicorn: Episode EX - 100 Years of Solitude
Mobile Suit Gundam I
Ring Ring Boy
All That Gundam
Mobile Suit Gundam
One Piece: Long Ring Long Land-hen
Mobile Suit Gundam-san (Movie)
Mobile Suit Gundam: Hathaway's Flash


Actual Anime Recommendation Tool

In [217]:
anime_name = input('Enter anime name : ')
title_list = anime_full.Name.to_list()
find_close_match = difflib.get_close_matches(anime_name, title_list)
anime_match = find_close_match[0]
index_of_anime = anime_full[anime_full.Name == anime_match].index.values[0]
similarity_score = list(enumerate(similarity[index_of_anime]))
most_similar_animes = sorted(similarity_score, key=lambda x:x[1], reverse=True)
print("15 Best Suggested Animes for your sunday night : \n")
for movie in most_similar_animes[:15]:
    print(anime_full[anime_full.index == movie[0]].Name.values[0])


15 Best Suggested Animes for your sunday night : 

Haikyuu!!
Haikyuu!!: To the Top
Haikyuu!! Second Season
Ashita e Attack!
Attack No.1 (1970)
Attacker You!
Attack No.1
Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou
2.43: Seiin Koukou Danshi Volley-bu
Shoujo Fight: Norainu-tachi no Odekake
Haikyuu!!: vs. "Akaten"
Haikyuu!!: To the Top 2nd Season
Moshidora
Tennis no Ouji-sama
Diamond no Ace


In [216]:
find_close_match

[]